In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# !pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
label2id = {'negative':0,'positive':1}

train = pd.read_csv(r'train.csv', encoding='utf_8_sig')
train = train.replace({"label": label2id})
train.fillna('', inplace=True)
test = pd.read_csv(r'test.csv', encoding='utf_8_sig')
test = test.replace({"label": label2id})
test.fillna('', inplace=True)

/tmp/ipykernel_9367/2093326092.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train = train.replace({"label": label2id})
/tmp/ipykernel_9367/2093326092.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test = test.replace({"label": label2id})


In [4]:
train

,label,text,diff
0,1,XSS 취약점 수정,diff --git a/adm/board_form_update.php b/adm/b...
1,0,"Auto merge of #4754 - daira:fix-enable-debug, ...",diff --git a/depends/hosts/darwin.mk b/depends...
2,1,CAMEL-9309: Make it easier to turn on|off java...,diff --git a/components/camel-ahc/src/main/jav...
3,1,Fixed bug #72627: Memory Leakage In exif_proce...,diff --git a/ext/exif/exif.c b/ext/exif/exif.c...
4,0,Merge branch 'dev',diff --git a/.htaccess b/.htaccess\nindex 3293...
...,...,...,...
7073,1,Restrict permissions on cinder configuration\n...,diff --git a/manifests/base.pp b/manifests/bas...
7074,1,avformat/avidec: Limit formats in gab2 to srt ...,diff --git a/libavformat/avidec.c b/libavforma...
7075,1,Fixed possible direct calls,diff --git a/phpmyfaq/admin/stat.ratings.php b...
7076,1,"Reject qname's wirelength > 255, `chopOff()` h...",diff --git a/pdns/dnsparser.cc b/pdns/dnsparse...


In [5]:
len(train)

7078

In [6]:
train['label'].value_counts()

label
0    4443
1    2635
Name: count, dtype: int64

In [7]:
test['label'].value_counts()

label
0    1904
1    1130
Name: count, dtype: int64

In [8]:
# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer
# model_id = "../../sentence-transformers/paraphrase-mpnet-base-v2"
model = SentenceTransformer('../../sentence-transformers/paraphrase-mpnet-base-v2')

In [9]:
train['embeddings'] = train['text'].apply(lambda x: model.encode(str(x)))
test['embeddings'] = test['text'].apply(lambda x: model.encode(str(x)))

In [10]:
test

,label,text,diff,embeddings
0,0,[mjr] Allow to optinoally specify end date for...,diff --git a/doc/CHANGES b/doc/CHANGES\nindex ...,"[-0.0950423, -0.049959756, -0.010932135, -0.10..."
1,1,Fixed memory leak when creating nested excepti...,diff --git a/ChangeLog b/ChangeLog\nindex 019b...,"[-0.0038684113, 0.0019372888, -0.07806292, 0.2..."
2,0,regenerate pyaff.c with a newer Cython\n\nCyth...,diff --git a/pyaff/pyaff.c b/pyaff/pyaff.c\nin...,"[-0.123547316, 0.039077245, 0.008539506, 0.026..."
3,1,Added some tests for CXF-4629\n\ngit-svn-id: h...,diff --git a/rt/ws/security/src/main/java/org/...,"[-0.17392713, 0.2297541, -0.044745788, 0.08347..."
4,1,Selectively cache headers\n\nCo-authored-by: J...,diff --git a/imap/util.c b/imap/util.c\nindex ...,"[0.023296649, 0.034457777, -0.04611981, 0.0770..."
...,...,...,...,...
3029,1,Add accessDeniedHandler method to ExceptionHan...,diff --git a/config/src/main/java/org/springfr...,"[-0.02450317, 0.45559555, 0.05852364, -0.00511..."
3030,0,Merge pull request #1247 from jbonofre/KARAF-6...,diff --git a/shell/ssh/src/main/java/org/apach...,"[-0.06588048, 0.12500134, 0.07387895, -0.05972..."
3031,0,Bump @popperjs/core from 2.5.4 to 2.6.0 in /fr...,diff --git a/frontend/package-lock.json b/fron...,"[-0.09501643, 0.032177404, -0.033742666, 0.047..."
3032,0,No one should care about ungifs.,diff --git a/README.md b/README.md\nindex fa21...,"[-0.07875047, 0.38262156, -0.0040256907, 0.030..."


In [11]:
X_train = train['embeddings'].to_list()
X_test = test['embeddings'].to_list()

y_train = train['label'].to_list()
y_test = test['label'].to_list()

In [12]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [13]:
from sklearn import metrics
predicted = LR.predict(X_test)
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.f1_score(y_test, predicted,average='weighted'))
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))

Logistic Regression Precision: 0.8542564621982858
Logistic Regression Recall: 0.8553065260382333
Logistic Regression Recall: 0.8542524722135222
Logistic Regression Accuracy: 0.8553065260382333
